In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from __future__ import print_function, division
from builtins import range

In [4]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from gensim.models import KeyedVectors

In [5]:
from nltk.corpus import stopwords


In [6]:
ignore_words = set(stopwords.words('english')).union(set(stopwords.words('indonesian')))
print(len(ignore_words))

936


In [10]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_cat=pd.read_json('categories.json')

In [11]:
df_train.title.head()



0                 nyx sex bomb pallete natural palette
1    etude house precious mineral any cushion pearl...
2                             milani rose powder blush
3                  etude house baby sweet sugar powder
4         bedak revlon color stay aqua mineral make up
Name: title, dtype: object

In [12]:
df_train['Group']=df_train.image_path.map(lambda x: x[:7])
df_test['Group']=df_test.image_path.map(lambda x: x[:7])

In [13]:
train,test=train_test_split(df_train,random_state=2019,stratify=df_train.Category,test_size=0.2)

In [14]:
np.random.seed(2019)

In [15]:
groups=['beauty_','mobile_','fashion']

In [16]:
# some linear models
from sklearn.linear_model import LogisticRegression, BayesianRidge

# SCM for classification
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

import warnings
warnings.filterwarnings('ignore')
np.random.seed(2019)


In [17]:
from sklearn.metrics import accuracy_score

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [19]:
# let's build pipeline for parameter search in n_grams and and various models
lr=LogisticRegression()
nb=MultinomialNB()
svc=SVC()
# let's check first three and see which one is best
gnb=GaussianNB()
bnb=BernoulliNB()
br=BayesianRidge()
rf=RandomForestClassifier(n_estimators=150)

In [52]:
# Optimized parameters are found by GridSearch

# Submit these results and see if it is really improved in leaderboard
pipe_beauty=Pipeline([('vectorize', CountVectorizer(ngram_range=(1,3))),('model',LogisticRegression(C=0.8))])
pipe_mobile=Pipeline([('vectorize', CountVectorizer(ngram_range=(1,3),min_df=2)),('model',LogisticRegression(C=1))])
pipe_fashion=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(1,5))),('model',LogisticRegression(C=0.4))])
pipe=[pipe_beauty,pipe_mobile,pipe_fashion]


groups=['beauty_','mobile_','fashion']
predictions_val={} # Use for ensembling model
predictions_proba_val={} # Use for ensembling model
predictions_all={} # check in submission file 
predictions_proba_all={} # Use for final ensembling
for i in tqdm(range(3)):
    print("Results for ", groups[i])
    model=pipe[i]
    X=train[train.Group==groups[i]].title
    y=train[train.Group==groups[i]].Category
    model.fit(X,y)
    predictions_val[groups[i]]=model.predict(test[test.Group==groups[i]].title)
    predictions_proba_val[groups[i]]=model.predict_proba(test[test.Group==groups[i]].title)
    print('Accuracy  ')
    print(accuracy_score(test[test.Group==groups[i]].Category,predictions_val[groups[i]]))

    
    model=pipe[i]
    X=df_train[df_train.Group==groups[i]].title
    y=df_train[df_train.Group==groups[i]].Category
    model.fit(X,y)
    predictions_all[groups[i]]=model.predict(df_test[df_test.Group==groups[i]].title)
    predictions_proba_all[groups[i]]=model.predict_proba(df_test[df_test.Group==groups[i]].title)
    


## Wow, interesting ! It turns out that stop words helps only for fashion category!
## Lets' list CV results for beauty, mobile  and fashion 
## beauty CV 0.784223069756
## Mobile CV 0.825934011102
## fashion CV 0.644946336401 with stop words

# Results with stop words.
# {'beauty_': 0.78344144628257784,
#  'mobile_': 0.8236574564959771,
#  'fashion': 0.6449463364011252}

# Our base model CV results. I should list them here



  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Results for  beauty_
Accuracy  
0.789800582724



 33%|███████████████████████████▋                                                       | 1/3 [08:48<17:36, 528.28s/it]

Results for  mobile_
Accuracy  
0.830687665679



 67%|███████████████████████████████████████████████████████▎                           | 2/3 [12:45<07:21, 441.03s/it]

Results for  fashion
Accuracy  
0.652056166223



100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [27:02<00:00, 565.79s/it]

In [53]:
classes={}

for i in range(3):
    classes[groups[i]]=pipe[i].classes_

In [54]:
classes

{'beauty_': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16], dtype=int64),
 'mobile_': array([31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57], dtype=int64),
 'fashion': array([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], dtype=int64)}

In [55]:
predictions_all['beauty_'].shape

(76545,)

In [56]:
predictions_val['beauty_'].shape

(57317,)

In [57]:
# Save all data both validation set and training set
# This data will be used for modeling ensembling and for final ensemble result
for i in range(3):
    # for ensembling
    df=pd.DataFrame(predictions_proba_all[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=df_test[df_test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_all_data2.csv',index=False)
    # for ensemble modeling
    df=pd.DataFrame(predictions_proba_val[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=test[test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_val_data2.csv',index=False)
